In [1]:
# input = """###############
# #.......#....E#
# #.#.###.#.###.#
# #.....#.#...#.#
# #.###.#####.#.#
# #.#.#.......#.#
# #.#.#####.###.#
# #...........#.#
# ###.#.#####.#.#
# #...#.....#.#.#
# #.#.#.###.#.#.#
# #.....#...#.#.#
# #.###.#.#.#.#.#
# #S..#.....#...#
# ###############"""
input = open("inputs/16").read()

In [2]:
input

'#############################################################################################################################################\n#.......#...#.......#.#.......................#.......#...#...#...#...............#.#.....................#.......#.......#.......#...#....E#\n#.#.###.#.#.###.###.#.#.###.#################.#.###.#.#.#.#.#.#.#.#.#######.#####.#.#.#.###.###.#####.#.#.#.#.#####.#.#####.#.###.#.#.#.###.#\n#.#.#.#.#.#.....#.#.........................#.#...#.#.#.....#...#.........#.#.....#...#.#.#...#.#.#...#.#...#.#...#.#.#.....#.#.#.#.#.....#.#\n###.#.#.#.#####.#.#.#.#####.#####.#.###.#.#.#.#.###.#.#####.###########.###.###.#######.#.###.#.#.#.###.#.#.#.#.#.#.#.#.#####.#.#.#########.#\n#...#.#...#.....#...#.#.....#.....#...#.#.#.#...#...#.....#...#...#.......#...#.........#...#.#.#.....#.......................#.#.........#.#\n#.###.#####.#####.###.#.###.#.###.#.#.#.#.#.#####.#######.#.###.#.#.###.#.###.###########.###.#.#####.###.#.#.#######.#.###.###.#########.#.#

In [3]:
import numpy as np


def parse_board(input):
    return np.array(list(map(list, input.splitlines())))


def add_tuple(t1, t2):
    return tuple(x + y for x, y in zip(t1, t2))


def print_board(board):
    print("\n".join("".join(row) for row in board))


dirs = {
    "^": (-1, 0),
    ">": (0, 1),
    "v": (1, 0),
    "<": (0, -1),
}

In [4]:
dir_neighbors = {
    "^": ["<", ">"],
    ">": ["^", "v"],
    "v": [">", "<"],
    "<": ["v", "^"],
}

In [5]:
board = parse_board(input)

In [6]:
start = tuple(int(i) for i in np.argwhere(board == "S")[0])
end = tuple(int(i) for i in np.argwhere(board == "E")[0])

# when we finish we're going to be either facing up or right. have to check both
ends = [(end, ">"), (end, "^")]
start = (start, ">")  # we start facing east

start, ends

(((139, 1), '>'), [((1, 139), '>'), ((1, 139), '^')])

In [7]:
board[ends[0][0]] = "."
board[start[0]] = "."

In [8]:
from collections import defaultdict
import heapq

distances = defaultdict(lambda: np.inf)
distances[start] = 0

pq = [(start, 0)]
heapq.heapify(pq)

best_paths = defaultdict(frozenset)
best_paths[start] = frozenset([start[0]])

while pq:
    current_node, current_distance = heapq.heappop(pq)

    # Skip if we've already found a better path
    if current_distance > distances[current_node]:
        continue

    # Process both forward movement and turns using a list of (next_node, cost) pairs
    possible_moves = []

    # Add forward movement if possible
    forward_pos = add_tuple(current_node[0], dirs[current_node[1]])
    if board[forward_pos] == ".":
        possible_moves.append(((forward_pos, current_node[1]), 1))

    # Add turns
    for turn_dir in dir_neighbors[current_node[1]]:
        possible_moves.append(((current_node[0], turn_dir), 1000))

    # Process all possible moves
    for neighbor, move_cost in possible_moves:
        new_distance = current_distance + move_cost

        if new_distance > distances[neighbor]:
            # nothing to do if this is a longer path
            pass
        elif new_distance < distances[neighbor]:
            distances[neighbor] = new_distance
            best_paths[neighbor] = best_paths[current_node] | frozenset([neighbor[0]])
            heapq.heappush(pq, (neighbor, new_distance))
        else:
            best_paths[neighbor] |= best_paths[current_node] | frozenset([neighbor[0]])
            heapq.heappush(pq, (neighbor, new_distance))

In [9]:
min(distances[e] for e in ends)

143580

In [10]:
combined_best = best_paths[ends[0]] | best_paths[ends[1]]
len(combined_best)

645

In [11]:
assert start[0] in combined_best
assert ends[0][0] in combined_best
assert ends[1][0] in combined_best